In [118]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse
from read_db.CH import Getch
import os

sns.set()


# Создадим вспомогательную функцию
def sign(x):
    """ Данная функция возвращает число со знаком """
    if x > 0:
        return '+' + str(x) + str('%')
    else:
        return str(x) + str('%')


def report(chat=None):
    """ Данная функция создает запрос в БД и присылает отчет по базовым метрикам в telegram"""
    chat_id = chat or 2093549033
    # bot = telegram.Bot(token=os.environ.get("REPORT_BOT_TOKEN"))  # шировка токена по пути Settings -> CI/CD -> Variables
    bot = telegram.Bot(token='**************************************************')

    
    query = '''
    SELECT toStartOfDay(toDateTime(day)) AS day_,
       sum(like) AS like,
       sum(view) AS view,
       sum(message) AS message
    FROM
      (select *
       from
         (select toDate(time) as day,
                 user_id,
                 countIf(user_id, action='like') as like,
                 countIf(user_id, action='view') as view
          from {db}.feed_actions
          group by day, user_id) t1
          
       FULL join
       
         (select toDate(time) as day,
                 user_id,
                 count(reciever_id) as message
          from simulator_20220120.message_actions
          group by day, user_id) t2 
          
        using day, user_id
        order by day, user_id) AS virtual_table
    WHERE day_ <= yesterday() and day_ > yesterday() - 8
    GROUP BY toStartOfDay(toDateTime(day))
    ORDER BY day_ DESC;
    '''
    #  FROM simulator_20220120.feed_actions
        
    
    data = Getch(query).df
    day = data.day_[0]
    view = data.view[0]
    view_to_day = round((data.view[0] / data.view[1] - 1) * 100, 2)
    view_to_week = round((data.view[0] / data.view[7] - 1) * 100, 2)
    
    like = data.like[0]
    like_to_day = round((data.like[0] / data.like[1] - 1) * 100, 2)
    like_to_week = round((data.like[0] / data.like[7] - 1) * 100, 2)
    
    message = data.message[0]
    message_to_day = round((data.message[0] / data.message[1] - 1) * 100, 2)
    message_to_week = round((data.message[0] / data.message[7] - 1) * 100, 2)
    

    
    # Динамика количества лайков, постов, сообщений за 1 день/неделю
    msg = ('Отчет за ' + str(day).split(' ')[0] + ':' + str('\n') +
           'Просмотры: ' + str('\n') + 
           str(view) + str(' / ') + sign(view_to_day) + str(' / ') + sign(view_to_week) + str('\n') + 
           
           'Лайки: '  + str('\n') +
           str(like) + str(' / ') + sign(view_to_day) + str(' / ') + sign(like_to_week) + str('\n') + 
           
           'Сообщения: ' + str('\n') +
           str(message) + str(' / ') + sign(message_to_day) + str(' / ') + sign(message_to_week) + str('\n') + 
           '--------------------------------------------------------------' + str('\n') +
           'кол-во/динамика за день/неделю')

    bot.sendMessage(chat_id=chat_id, text= msg)
    
 

    # Отправка графиков по базовым метрикам
    y = data.iloc[:7, [2,1,3]]
    x = data.iloc[:7, 0]
    plt.title('Динамика количества лайков, постов, сообщений')
    plt.xticks(rotation=15)
    plt.plot(x,y)
    plt.legend(['views', 'likes', 'messages'], )

    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.name = 'plot.png'
    plot_object.seek(0)
    bot.sendPhoto(chat_id = chat_id, photo = plot_object)
    plt.close()

    

    query2 = '''
    SELECT post_id AS post_id,
       countIf(action = 'view') AS "view",
       countIf(action = 'like') AS "like",
       countIf(action = 'like') / countIf(action = 'view') AS "CTR",
       count(DISTINCT user_id) AS "courage"
    FROM {db}.feed_actions
    WHERE toDate(time) = yesterday()
    GROUP BY post_id
    ORDER BY "view" DESC
    LIMIT 10;
    '''
    #  FROM simulator_20220120.feed_actions

    
    data2 = Getch(query2).df
    file_object = io.StringIO()
    data2.to_csv(file_object, index=False)
    name_file = str(day).split(' ')[0] + ' TOP_10_posts' + '.csv'
    file_object.name = name_file
    file_object.seek(0)
    bot.sendDocument(chat_id=chat_id, document=file_object)


try:
    report()
except Exception as e:
    print(e)

137059 133540 107670
